In [2]:
import numpy as np
#from scipy.signal import butter, filtfilt
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
#import seaborn as sns
#sns.set_context("notebook", font_scale=1.3,
#                rc={'lines.linewidth': 1.5, 'lines.markersize': 14, 'axes.titlesize': 'x-large'})
#matplotlib.rc('legend', numpoints=1, fontsize=14)
#import glob
import sys, os
sys.path.insert(1, r'./Functions')
# IPython widgets:
from IPython.display import display
#%pip install ipywidgets
#jupyter labextension install @jupyter-widgets/jupyterlab-manager
#import ipywidgets
#from ipywidgets import FloatProgress, interactive
import seaborn as sns
from scipy.signal import detrend
import pickle

In [3]:
%pip install version_information
%load_ext version_information
%version_information numpy, scipy, pandas, matplotlib, seaborn


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Software versions
Python 3.10.4 64bit [GCC 9.4.0]
IPython 8.12.0
OS Linux 5.15.0 1038 azure x86_64 with glibc2.31
numpy 1.24.3
scipy 1.10.1
pandas 2.0.1
matplotlib 3.7.1
seaborn 0.12.2
Thu Jun 01 18:33:00 2023 UTC

In [4]:
# GitHub URL:
path2 = 'https://raw.githubusercontent.com/demotu/datasets/master/PDS/data'
#local directory:
#path2 = r"C:\Users\ferna\Downloads\IC-dados\Dados"
#path2 = '/home/rnwatanabe/copdata'
fname = os.path.join(path2, 'PDSinfo.txt')
PDSinfo = pd.read_csv(fname, sep='\t', header=0, index_col=None, engine='c', encoding='utf-8')
print(fname)
print("Information of %s subjects loaded (%s rows, %s columns)."
      %(len(pd.unique(PDSinfo.Subject)), PDSinfo.shape[0], PDSinfo.shape[1]))

https://raw.githubusercontent.com/demotu/datasets/master/PDS/data/PDSinfo.txt
Information of 49 subjects loaded (588 rows, 29 columns).


In [5]:
a=PDSinfo[PDSinfo['Subject']==44].index.tolist()
e=PDSinfo[PDSinfo['Subject']==44].index.tolist()[2]
ei = PDSinfo[PDSinfo['Subject']==44]
PDSinfo.memory_usage()

Index                 128
Trial                4704
Subject              4704
Vision               4704
Surface              4704
Rep                  4704
Age                  4704
AgeGroup             4704
Gender               4704
Height               4704
Mass                 4704
BMI                  4704
FootLen              4704
DominantLeg          4704
Nationality          4704
SkinColor            4704
Ystudy               4704
Footwear             4704
Illness              4704
Illness2             4704
Nmedication          4704
Medication           4704
Ortho-Prosthesis     4704
Ortho-Prosthesis2    4704
Disability           4704
Disability2          4704
Falls12m             4704
PhysicalActivity     4704
Sequence             4704
Date                 4704
dtype: int64

In [6]:
fs=100

In [7]:
file = open('filename','rb')
(COPXreal,COPZreal,COPX,COPZ,d1x,d1z,d2x,d2z,matrizangx,matrizangz,COGX,COGY,COGZ,angulox,anguloz,d,alpha,subject,ei,sinx,sinz,cosx,cosz,tanx,tanz,COGXtrain,COGYtrain,COGZtrain,COGXtest,COGYtest,COGZtest,COPXtrain,COPXtest,COPZtrain,COPZtest,train,test,masstrain,masstest,subjectp,subjectc)= pickle.load(file)
file.close()

In [8]:
filename = open('file_name','rb')


(COPXreal,COPZreal,COPX,COPZ,d1x,d1z,d2x,d2z,matrizangx,matrizangz,ASISX,ASISY,ASISZ,angulox,anguloz,d,alpha,subject,ei,sinx,sinz,cosx,cosz,tanx,tanz,ASISXtrain,ASISYtrain,ASISZtrain,ASISXtest,ASISYtest,ASISZtest,COPXtrain,COPXtest,COPZtrain,COPZtest,train,test,masstrain,masstest,subjectp,subjecta)= pickle.load(filename)

filename.close()

In [12]:
def newdist(paramt,x,y,z):
    from sklearn.model_selection import train_test_split
    from scipy.optimize import minimize, rosen, rosen_der
    from sklearn.metrics import mean_squared_error
    dest = np.zeros((60*fs-30))
    d = np.zeros((60*fs-30))
    for i in range(0, len(x)):

        delta,mi = paramt
        
        d[i] = np.mean(np.sqrt(x[i]**2+y[i]**2+z[i]**2))
        
        dest[i] = delta * d[i] + mi
    
      
    return dest


In [13]:
#for i in range(0,len(COGXtrain)):
        
        #d= np.sqrt(COGXtrain[i]**2+COGYtrain[i]**2+COGZtrain[i]**2)
        
def meansquarerrordist(paramt,d,ASISXtrain,ASISYtrain,ASISZtrain):
    d = np.zeros((60*fs-30))
    from sklearn.metrics import mean_squared_error
    ytrue = np.zeros((60*fs-30))
    ypredicted = np.zeros((60*fs-30))
    for i in range(0,len(ASISXtrain)):
        
        d[i] = np.mean(np.sqrt(COGXtrain[i]**2+COGYtrain[i]**2+COGZtrain[i]**2))
        
        ytrue[i] = d[i]
        
        ypredicted = newdist(paramt,ASISXtrain,ASISYtrain,ASISZtrain)
        
        
    return mean_squared_error(ytrue, ypredicted)


In [14]:
from sklearn.model_selection import train_test_split
#from scipy import optmize
from scipy.optimize import minimize, rosen, rosen_der
first_guess = [1,1]
res = minimize(meansquarerrordist, first_guess,(d,COGXtrain,COGYtrain,COGZtrain))
coeficientes = res.x
delta=coeficientes[0]
mi=coeficientes[1]
print(delta)
print(mi)

NameError: name 'COGYtrain' is not defined

In [ ]:
treino = newdist(coeficientes,COGXtrain,COGYtrain,COGZtrain)
teste = newdist(coeficientes,COGXtest,COGYtest,COGZtest)

Regressão linear distância

In [ ]:
def centerofpressure(paramt, M,x,y,z):
    from sklearn.model_selection import train_test_split
    from scipy.optimize import minimize, rosen, rosen_der
    from sklearn.metrics import mean_squared_error
    COPXES = np.empty((49*12, 60*fs-30))
    COPXESorig =np.empty((49*12, 60*fs-30))
    dest =np.empty((49*12,60*fs-30))
    trial = 0 
    sennovox=np.empty((49*12,60*fs-30))
    cosnovox =np.empty((49*12,60*fs-30))
    sennovoz=np.empty((49*12,60*fs-30))
    cosnovoz =np.empty((49*12,60*fs-30))
    
    for i in range(0,len(x)):
        
        A,alpha,beta, gama= paramt
        
        d[trial,:] = np.mean(np.sqrt(x[trial,:]**2+y[trial,:]**2+z[trial,:]**2))
        
        dest[trial,:] = delta * d[trial,:] + mi
        
        angulox[trial,:] = np.arcsin(np.tanh(x[trial,:]/dest[trial,:]))
        anguloz[trial,:] = np.arcsin(np.tanh(z[trial,:]/dest[trial,:]))
        sinx=np.sin(angulox[trial,:])
        sinz=np.sin(anguloz[trial,:])
        cosx=np.cos(angulox[trial,:])
        cosz=np.cos(anguloz[trial,:])
        tanx=np.tan(angulox[trial,:])
        tanz=np.tan(anguloz[trial,:])
            
        matrizangx[trial,:] = angulox[trial,:]
        matrizangz[trial,:] = anguloz[trial,:]







        sennovox[trial,:] = (sinx)*A + ((1-A**2)**1/2)*(cosx)
            

        cosnovox[trial,:] = (cosx)*A - ((1-A**2)**1/2)*(sinx)
            
        sennovoz[trial,:] = (sinz)*A + ((1-A**2)**1/2)*(cosz)
           

        cosnovoz[trial,:] = (cosz)*A - ((1-A**2)**1/2)*(sinz)

        



        Jb=(4/3)*(M[trial])*((dest[trial,:])**2)
            
        t=grf['Time']
        dt=t[1]-t[0]
            
        d1x[trial,:]=np.gradient(matrizangx[trial,:],dt)
        d1z[trial,:]=np.gradient(matrizangz[trial,:],dt)
            
            
        d2x[trial,:]=np.gradient(d1x[trial,:],dt)
        d2z[trial,:]=np.gradient(d1z[trial,:],dt)
            
        COPX[trial,:] = dest[trial,:]*sennovox[trial,:] + Jb/(M[trial]*9.8)*(sennovox[trial,:]*d1x[trial,:]**2-cosnovox[trial,:]*d2x[trial,:])
        COPZ[trial,:] = dest[trial,:]*sennovoz[trial,:] + Jb/(M[trial]*9.8)*(sennovoz[trial,:]*d1z[trial,:]**2-cosnovoz[trial,:]*d2z[trial,:])

        COPXES[trial,:]= alpha*(dest[trial,:]*np.tanh(sennovox[trial,:]) + ((Jb/(M[trial]*9.8))*np.tanh(sennovox[trial,:])*d1x[trial,:]**2)) + d2x[trial,:]*(-alpha*np.tanh(cosnovox[trial,:])*Jb/(M[trial]*9.8)+beta) + gama
        COPXESorig[trial,:]= COPX[trial,:]
        
        
        trial = trial +1
        
    
    return COPXES



In [ ]:
def meansquarerror(paramt,COPXtrain,masstrain,ASISXtrain,ASISYtrain,ASISZtrain):
    
    from sklearn.metrics import mean_squared_error
    
    ytrue = np.empty((49*12, 60*fs-30))
    A,alpha,beta, gama = paramt
    for i in range(0,len(ASISXtrain)):
        
        ytrue[i] = COPXtrain[i]
    
        ypredicted = centerofpressure(paramt,masstrain,ASISXtrain,ASISYtrain,ASISZtrain)
    
        
    
    
    return mean_squared_error(ytrue, ypredicted)
        


    

In [ ]:
from sklearn.model_selection import train_test_split
#from scipy import optmize
from scipy.optimize import minimize, rosen, rosen_der
first_guess = [1,1,1,1]
res = minimize(meansquarerror, first_guess,(COPXtrain,masstrain,ASISXtrain,ASISYtrain,ASISZtrain))
coeficientes = res.x
A=coeficientes[0]
alpha=coeficientes[1]
beta=coeficientes[2]
gama=coeficientes[3]

Test

In [ ]:

treinando=centerofpressure(coeficientes, masstrain,ASISXtrain,ASISYtrain,ASISZtrain) 

testando=centerofpressure(coeficientes, masstest,ASISXtest,ASISYtest,ASISZtest)


In [ ]:
time = np.arange(0,6000)/100
for i in range(0,len(ASISXtest)): 
    plt.figure()
    plt.plot(time[:5970],detrend(testando[i,:]),color='purple',label='COP predicted')
    plt.plot(time[:5970],detrend(COPXreal[i]),color='blue', label='COP real')
    plt.title('COP real X COP calculated for test {:.1f}'.format(i))
    plt.xlabel('Time')
    plt.legend()
    plt.show()

In [ ]:
sennovo = sinx*A + ((1-A**2)**1/2)*cosx
print(np.max(sennovo))
print(np.min(sennovo))

In [ ]:
cosnovo = cosx*A - ((1-A**2)**1/2)*sinx
print(np.max(cosnovo))
print(np.min(cosnovo))